# multi_columnsの動作確認(3行ヘッダー版)

In [1]:
from IPython.display import display
import pandas as pd
import sys
from os.path import abspath, split, join, dirname, realpath
from inspect import getfile, currentframe
pjc_folder = realpath(dirname(abspath(split(getfile( currentframe() ))[0])))

if pjc_folder not in sys.path:
    sys.path.insert(0, pjc_folder)
pass

In [2]:
# 実行上必要なものではないです(GitHubにあげる上で、ファイルパスなどを消すためにワーニングを非表示)
import warnings
warnings.filterwarnings('ignore')

In [3]:
xl_file = 'test_col3.xlsx'
xi_path = join(pjc_folder, 'file' , xl_file)

In [4]:
#まずは普通にpandasのread_excelを使って読む
df = pd.read_excel(xi_path, skiprows=2, header=[0,1,2])
display(df)

測定      測定                               プログラム                     \
No     その他                  Unnamed: 3_level_1 Unnamed: 4_level_1   
        体重  体脂肪率        BMI                 C#              ｐｙてょｎ   
1002  65.5  19.5  20.216049                1.0                2.0   
1003  66.0  18.1  20.370370                NaN                NaN   
1004  65.7  19.1  20.277778                NaN                0.5   
1005  65.8  19.3  20.308642                NaN                7.0   
1006  65.3  20.1  20.154321                1.0                6.0   

測定                   備考  
No   Unnamed: 5_level_1  
     Unnamed: 5_level_2  
1002                NaN  
1003               飲み過ぎ  
1004                NaN  
1005                NaN  
1006                NaN

In [5]:
# 次に作成したプログラムを読み込んで、multi_columns関数に食わせて整形
from convert_excel import multi_columns
df = multi_columns(df)
display(df)

測定                        プログラム          備考
     No   その他                     C# ｐｙてょｎ      
           体重  体脂肪率        BMI                  
0  1002  65.5  19.5  20.216049   1.0   2.0   NaN
1  1003  66.0  18.1  20.370370   NaN   NaN  飲み過ぎ
2  1004  65.7  19.1  20.277778   NaN   0.5   NaN
3  1005  65.8  19.3  20.308642   NaN   7.0   NaN
4  1006  65.3  20.1  20.154321   1.0   6.0   NaN

In [6]:
df.columns.names

FrozenList([None, None, None])

In [7]:
list(df.columns)

[('測定', 'No', ''),
 ('測定', 'その他', '体重'),
 ('測定', 'その他', '体脂肪率'),
 ('測定', 'その他', 'BMI'),
 ('プログラム', 'C#', ''),
 ('プログラム', 'ｐｙてょｎ', ''),
 ('備考', '', '')]

### ここから色々操作してみる

In [8]:
# まずは子の項目のないものから
df["備考"]

0     NaN
1    飲み過ぎ
2     NaN
3     NaN
4     NaN
Name: 備考, dtype: object

In [9]:
# Noは作られた直後はindexではない。普通の列項目だが、測定項目の子項目なので測定の指定が必要
df["測定", "No"]

0    1002
1    1003
2    1004
3    1005
4    1006
Name: (測定, No), dtype: int64

In [10]:
# 階層の深いもの(複数キーがないと断定できないもの)
df["測定", "その他", "BMI"]

0    20.216049
1    20.370370
2    20.277778
3    20.308642
4    20.154321
Name: (測定, その他, BMI), dtype: float64

In [11]:
# 状態だけを選ぶと子の要素二つが選ばれる。パフォーマンスの警告が出るが実施可能。
df["測定", "その他"]

,体重,体脂肪率,BMI
0,65.5,19.5,20.216049
1,66.0,18.1,20.370370
2,65.7,19.1,20.277778
3,65.8,19.3,20.308642
4,65.3,20.1,20.154321


In [12]:
# 階層が2つの項目にもアクセスできる
df["プログラム", "C#"]

0    1.0
1    NaN
2    NaN
3    NaN
4    1.0
Name: (プログラム, C#), dtype: float64

In [13]:
# No列をインデックスにする場合はチョット注意が必要。特にインデックス後に削除する場合はタプルで指定する。
df.index = df["測定", "No"]
df.index.name = "No"
df = df.drop(columns=("測定", "No"))
display(df)

測定                  プログラム          備考
       その他                     C# ｐｙてょｎ      
        体重  体脂肪率        BMI                  
No                                           
1002  65.5  19.5  20.216049   1.0   2.0   NaN
1003  66.0  18.1  20.370370   NaN   NaN  飲み過ぎ
1004  65.7  19.1  20.277778   NaN   0.5   NaN
1005  65.8  19.3  20.308642   NaN   7.0   NaN
1006  65.3  20.1  20.154321   1.0   6.0   NaN

In [14]:
# 列を選択すると、Noがキーとして出力されている
df["備考"]

No
1002     NaN
1003    飲み過ぎ
1004     NaN
1005     NaN
1006     NaN
Name: 備考, dtype: object